<a href="https://colab.research.google.com/github/slightperturbation/ml_examples/blob/master/ML_Examples_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning Inception

This example shows using a pretrained (on imagenet) Inception V3 model to classify CIFAR images.

The Inception model is given an extra fully connected layer at the end for the classification.

Input for the prebuild inception model is 75x75, but CIFAR images are 32x32, so a resize layer is added to the beginning of the standard model.

In [1]:
import os
import tensorflow as tf

In [2]:
# Download the model and the weights
inception = tf.keras.applications.inception_v3.InceptionV3(input_shape = (75, 75, 3),
                        include_top = False)

# Lock it so it doesn't update during backpropagation
for layer in inception.layers:
  layer.trainable = False

#inception.summary()

Add our own trainable layers on top of the pre-trained CNNs.

In [3]:
# Final layers to learn the classifier

# 'mixed7' is the final CNN layer that we'll build on-top of.
x = tf.keras.layers.Flatten()(inception.get_layer('mixed7').output)
x = tf.keras.layers.Dense(1024, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.2)(x)                  
x = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(x)

finetuned_inception = tf.keras.Model(inception.input, x)

# Add a new layer at the beginning to resize the image up to Inception size.
input = tf.keras.Input(shape=(32, 32, 3))
preprocess = tf.keras.layers.experimental.preprocessing.Resizing(75, 75)(input)
preprocess = finetuned_inception(preprocess)

model = tf.keras.Model(input, preprocess)

model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)
tf.keras.utils.plot_model(model.get_layer('functional_1'), show_shapes=True)

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
resizing (Resizing)          (None, 75, 75, 3)         0         
_________________________________________________________________
functional_1 (Functional)    (None, 10)                16064426  
Total params: 16,064,426
Trainable params: 7,089,162
Non-trainable params: 8,975,264
_________________________________________________________________


In [4]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [5]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0


In [6]:
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

Epoch 1/10
1563/1563 [==============================] - 26s 17ms/step - loss: 1.7571 - accuracy: 0.7136 - val_loss: 1.7140 - val_accuracy: 0.7510
Epoch 2/10
1563/1563 [==============================] - 25s 16ms/step - loss: 1.6813 - accuracy: 0.7836 - val_loss: 1.6893 - val_accuracy: 0.7744
Epoch 3/10
1563/1563 [==============================] - 25s 16ms/step - loss: 1.6538 - accuracy: 0.8105 - val_loss: 1.6761 - val_accuracy: 0.7857
Epoch 4/10
1563/1563 [==============================] - 25s 16ms/step - loss: 1.6346 - accuracy: 0.8302 - val_loss: 1.6732 - val_accuracy: 0.7882
Epoch 5/10
1563/1563 [==============================] - 25s 16ms/step - loss: 1.6198 - accuracy: 0.8446 - val_loss: 1.6764 - val_accuracy: 0.7859
Epoch 6/10
1563/1563 [==============================] - 25s 16ms/step - loss: 1.6070 - accuracy: 0.8577 - val_loss: 1.6657 - val_accuracy: 0.7961
Epoch 7/10
1563/1563 [==============================] - 25s 16ms/step - loss: 1.5945 - accuracy: 0.8711 - val_loss: 1.6632 -